In [1]:
import numpy as np
import pandas as pd
from ucimlrepo import fetch_ucirepo 

In [2]:
dataset_ids = {
  'yeast':110, 
  'iris': 53, 
  'beans':602, 
  'rice': 545, 
  'raisins': 850,
  'wine': 109,
  'wine_quality': 186,
  'abalone': 1,
  'glass': 42,
  'obesity': 544
}

In [3]:
datasets = {}
variables = {}
targets = {}

for dataset_name, dataset_id in dataset_ids.items():
  repo = fetch_ucirepo(id=dataset_id)
  datasets[dataset_name] = repo.data.features
  variables[dataset_name] = repo.variables
  targets[dataset_name] = repo.data.targets

# Filters data that isn't continuous
for dataset_name, data in variables.items():
  to_drop = data[(data['type'] != 'Continuous') | (data['role'] != 'Feature')]['name'].to_list()

  for column in to_drop:
    try:
      datasets[dataset_name] = datasets[dataset_name].drop(column, axis='columns')
    except KeyError:
      pass

Column Sequence_Name not found in yeast
Column localization_site not found in yeast


,mcg,gvh,alm,mit,erl,pox,vac,nuc
0,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22
1,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22
2,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22
3,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22
4,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22


Column class not found in iris


,sepal length,sepal width,petal length,petal width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


Column Class not found in beans


...

,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055


Column Id_number not found in glass
Column Type_of_glass not found in glass


,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0


Column NObeyesdad not found in obesity


,Age,Height,Weight,NCP,CH2O,FAF
0,21.0,1.62,64.0,3.0,2.0,0.0
1,21.0,1.52,56.0,3.0,3.0,3.0
2,23.0,1.80,77.0,3.0,2.0,2.0
3,27.0,1.80,87.0,3.0,2.0,2.0
4,22.0,1.78,89.8,1.0,2.0,0.0


In [12]:
for dataset_name, data in datasets.items():
  centers = targets[dataset_name].iloc[:, 0].nunique()

  with open('real-instances/' + dataset_name + '.dat', 'w') as f:
    f.write(f"CENTERS:{centers}\n")
    # f.write(f"COV:{args.cov_list}\n") # N sei oq fz com isso aq
    # f.write(f"POINTS_PER_CENTER:{0}\n") # Acho que isso n se aplica aq
    # f.write(f"CENTERS_COORDS:{args.centers}\n") # Isso a gente não sabe a princípio nos dados reais
    f.write(f"POINTS/LABELS: \n")

    for index, row in data.iterrows():
      f.write(';'.join([str(x) for x in row]) + ';' + str(targets[dataset_name].iloc[index, 0]) + '\n')